<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Kian Omoomi
### Student Number : 98110062
### Colab Link: 
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [7]:
!wget "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-05 01:39:04--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-05 01:39:05--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1d2abe220c0a216577ebf97bec.dl.dropboxusercontent.com/cd/0/get/B7ZM1tprHjstrYtT-sYRSS-WDXJ8M1Eqs-rcvgnrq6G0XqbmsMExCopSfc3J-BCX9LCWa69sswuKVvZE7uGBC1-13itRghEKaoOfQdtAF1XXdWL1iXd8ms5x-nSKnAO680KsDoA91fYyxVYucpdrR25iN8V9_PiO3qjfARTlvT0IIoMcjE_W0Iy6C5gRHDxIfpk/file?dl=1# [following]
--2023-05-05 01:39:05--  https://uc1d2abe220c0a216577ebf97bec.dl.dropboxusercontent.com/cd/0/get/B7ZM1tprHjstrYtT-sYRSS-WDXJ8M1Eqs-rcvgnrq6G0XqbmsMExCopSfc3J-BCX9LCW

 18500K .......... .......... .......... .......... .......... 11% 4.36M 7m30s
 18550K .......... .......... .......... .......... .......... 11%  128M 7m29s
 18600K .......... .......... .......... .......... .......... 11% 67.0M 7m28s
 18650K .......... .......... .......... .......... .......... 11% 58.2M 7m26s
 18700K .......... .......... .......... .......... .......... 11% 85.3K 7m30s
 18750K .......... .......... .......... .......... .......... 11% 2.77M 7m28s
 18800K .......... .......... .......... .......... .......... 11%  129K 7m30s
 18850K .......... .......... .......... .......... .......... 11% 68.8M 7m29s
 18900K .......... .......... .......... .......... .......... 11%  101M 7m27s
 18950K .......... .......... .......... .......... .......... 11% 58.8M 7m26s
 19000K .......... .......... .......... .......... .......... 11%  215K 7m27s
 19050K .......... .......... .......... .......... .......... 11% 38.2M 7m25s
 19100K .......... .......... .......... .......... 

In [5]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

The syntax of the command is incorrect.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


# Preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [3]:
df = pd.read_csv('content/news-train.csv')
df

,topic,text
0,sports,به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1,politics,پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2,politics,# دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3,economics,کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4,economics,به گزارش ایسنا صحبت با برخی # # آن است که امرو...
...,...,...
197356,sports,به گزارش ورزش سه و به نقل از مارکا رقابت‌های ب...
197357,sports,به گزارش ایسنا و به نقل از فوتبال ایتالیا یوون...
197358,sports,سرمربی تیم والیبال دانشجویان گفت ما اصلا فرصت ...
197359,sports,به گزارش ورزش سه مرتضی # باشگاه # مصر # را نسب...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [4]:
df['topic'].value_counts()

topic
sports       123105
economics     49600
cultural      13359
politics      11297
Name: count, dtype: int64

Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [5]:
min_count = 11297
df_sports = df[df['topic'] == 'sports'].sample(min_count, random_state=42)
df_economics = df[df['topic'] == 'economics'].sample(min_count, random_state=42)
df_cultural = df[df['topic'] == 'cultural'].sample(min_count, random_state=42)
df_politics = df[df['topic'] == 'politics']
df = pd.concat([df_sports, df_economics, df_cultural, df_politics])
df = df.reset_index(drop=True)
df

,topic,text
0,sports,عباس آقایی در نشست خبری پیش از دیدار تیمش مقاب...
1,sports,بانوان کشتی‌گیر ایرانی هم خود را به دنیا معرفی...
2,sports,به گزارش ایسنا و به نقل از # # # سرمربی فرانسو...
3,sports,به گزارش ورزش سه یکی از گزینه‌های مهدی رحمتی ب...
4,sports,تیم ملی والیبال ایران در سومین دیدار خود از هف...
...,...,...
45183,politics,آیین # و # فرمانده نیروی دریایی سپاه دیروز با ...
45184,politics,رئیس‌جمهور با اشاره به تلاش‌های انجام شده برای...
45185,politics,عضو مجمع تشخیص مصلحت نظام گفت در حال حاضر حاکم...
45186,politics,پس از پایان دور ششم مذاکرات وین طرفین به مثبت ...


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [6]:
train_df, test_df = train_test_split(df, test_size=0.2)


In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [7]:
train_df['tokens'] = train_df['text'].apply(lambda x: x.strip().split(' '))

In [8]:
train_df

,topic,text,tokens
29750,cultural,معاون قرآن و # وزارت فرهنگ و ارشاد اسلامی گفت ...,"[معاون, قرآن, و, #, وزارت, فرهنگ, و, ارشاد, اس..."
20187,economics,تشکیل جلسه # طبق یک # # هفته‌های پایانی هر سال...,"[تشکیل, جلسه, #, طبق, یک, #, #, هفته‌های, پایا..."
35162,politics,روز گذشته رئیس‌جمهور در جلسه هیئت دولت با بیان...,"[روز, گذشته, رئیس‌جمهور, در, جلسه, هیئت, دولت,..."
32444,cultural,به گزارش ایسنا طبق گزارش رسیده مهدی شفیعی مدیر...,"[به, گزارش, ایسنا, طبق, گزارش, رسیده, مهدی, شف..."
41097,politics,تلاش ایالات متحده برای # به ائتلاف نظامی دریای...,"[تلاش, ایالات, متحده, برای, #, به, ائتلاف, نظا..."
...,...,...,...
19871,economics,به گزارش ایسنا # در جلسه شورای اداری اعضا اتاق...,"[به, گزارش, ایسنا, #, در, جلسه, شورای, اداری, ..."
24294,cultural,# # و نامناسب همچنان ساخته و نصب می‌شوند مثل ت...,"[#, #, و, نامناسب, همچنان, ساخته, و, نصب, می‌ش..."
37692,politics,اگرچه قرار نبود اما ایران و برجام در جلسه روز ...,"[اگرچه, قرار, نبود, اما, ایران, و, برجام, در, ..."
20468,economics,رئیس مرکز پژوهش‌های مجلس شورای اسلامی عنوان کر...,"[رئیس, مرکز, پژوهش‌های, مجلس, شورای, اسلامی, ع..."


Count the tokens and list the top 5 frequent ones (5 points)

In [9]:
from collections import Counter
token_counts = Counter([token for tokens in train_df['tokens'] for token in tokens])
most_common_tokens = token_counts.most_common(5)
for token, count in most_common_tokens:
    print(f"{token}: {count}")


#: 1616330
و: 874258
در: 741132

: 588664
به: 579704


Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [12]:
!wget stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-05 03:28:19--  http://stopwords.txt/
Resolving stopwords.txt (stopwords.txt)... failed: No such host is known. .
wget: unable to resolve host address 'stopwords.txt'
--2023-05-05 03:28:20--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-05 03:28:20--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length

In [11]:
file = open('stopwords.txt', 'r', encoding='utf-8')
stopwords = file.read().split('\n')
stopwords.append('#')

In [12]:
train_df['tokens'] = train_df['tokens'].apply(lambda tokens: [token for token in tokens if token not in stopwords])

Make a list of the remaining unique tokens (5 points)

In [13]:
unique_tokens = list(set([token for tokens in train_df['tokens'] for token in tokens]))

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [14]:
vectorizer = CountVectorizer(vocabulary=unique_tokens)
term_doc_matrix = vectorizer.fit_transform(train_df['tokens'].apply(lambda tokens: ' '.join(tokens)))

In [15]:
term_doc_matrix = pd.DataFrame(term_doc_matrix.toarray(), columns=vectorizer.get_feature_names_out())
term_doc_matrix

,تعاونی‌های,این,آتی,تصویری,استراماچونی,داوود,مهندس,ایران,عیسی,اکران,...,مهمان,عام,ایستاد,استادیوم,می‌دهیم,تیم‌ها,اینستاگرام,بانوان,شفیعی,ملاحظه
0,0,13,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,33,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,7,0,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,42,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,0
4,0,24,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36145,0,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36146,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36147,0,28,0,0,0,0,0,44,0,0,...,0,0,0,0,0,0,0,0,0,0
36148,0,17,0,0,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_train = term_doc_matrix.to_numpy()
X_train

array([[ 0, 13,  0, ...,  0,  0,  0],
       [ 0, 33,  0, ...,  0,  0,  0],
       [ 0,  7,  0, ...,  0,  0,  0],
       ...,
       [ 0, 28,  0, ...,  0,  0,  0],
       [ 0, 17,  0, ...,  0,  0,  0],
       [ 0, 38,  0, ...,  0,  0,  0]])

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [17]:
le = LabelEncoder()
y_train = le.fit_transform(train_df['topic'])
# cultural-0, economics-1, politics-2, sports-3
y_train

array([0, 1, 2, ..., 2, 1, 2])

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [18]:
test_df['tokens'] = test_df['text'].apply(lambda x: x.split(' '))
test_df['tokens'] = test_df['tokens'].apply(lambda tokens: [token for token in tokens if token not in stopwords])
vectorizer = CountVectorizer(vocabulary=unique_tokens)
term_doc_matrix_test = vectorizer.fit_transform(test_df['tokens'].apply(lambda tokens: ' '.join(tokens)))
term_doc_matrix_test = pd.DataFrame(term_doc_matrix_test.toarray(), columns=vectorizer.get_feature_names_out())
X_test = term_doc_matrix_test.to_numpy()
X_test

array([[ 0, 29,  0, ...,  0,  0,  0],
       [ 0, 22,  0, ...,  0,  0,  0],
       [ 0, 17,  0, ...,  0,  0,  0],
       ...,
       [ 0,  7,  0, ...,  0,  0,  0],
       [ 0, 20,  0, ...,  0,  0,  0],
       [ 0, 26,  0, ...,  0,  0,  2]])

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [19]:
y_test = le.transform(test_df['topic'])
y_test

array([0, 0, 2, ..., 1, 2, 2])

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [20]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

Get the predictions of the model for the testing news (`x_test`).

In [21]:
y_pred = clf.predict(X_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95      2263
           1       0.92      0.93      0.93      2258
           2       0.95      0.92      0.93      2267
           3       1.00      0.99      0.99      2250

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [27]:
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        """
        YOUR CODE
        """
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        
        """
        YOUR CODE
        """
        y_pred = [self.helper_predict(x) for x in X]
        return np.array(y_pred)
    
    def pdf(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx] + 1e-4
        numerator = np.exp(-(x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
    def helper_predict(self, x):
        posteriors = []
        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            class_conditional = np.sum(np.log(self.pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]
    


Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [28]:
my_clf = Naive_Bayes()
my_clf.fit(X_train, y_train)

Get the predictions of your model for the testing news (`x_test`).

In [29]:
my_y_pred = my_clf.predict(X_test)

/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_90408/541295603.py:62: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self.pdf(idx, x)))
/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_90408/541295603.py:62: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self.pdf(idx, x)))
/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_90408/541295603.py:62: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self.pdf(idx, x)))
/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_90408/541295603.py:62: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self.pdf(idx, x)))
/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_90408/541295603.py:62: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self.pdf(idx, x)))
/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_90408/

KeyboardInterrupt: 

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [26]:
print(classification_report(y_test, my_y_pred))

              precision    recall  f1-score   support

           0       0.64      0.94      0.76      2263
           1       0.83      0.81      0.82      2258
           2       0.92      0.61      0.73      2267
           3       0.92      0.82      0.87      2250

    accuracy                           0.80      9038
   macro avg       0.83      0.80      0.80      9038
weighted avg       0.83      0.80      0.80      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef